In [ ]:
import urllib.request
from pathlib import Path
from typing import List, NamedTuple, Optional
import os
import pandas as pd
from datetime import  datetime
from PIL import Image as im
import cv2

import numpy as np

import streamlit as st

import base64
db_name = 'face_rec_attendence1'
##
import face_recognition as face_rec
import cv2
import shutil
path = 'employee images'
employeeImg = []

employeeName = []
myList = os.listdir(path)
filename = 'click'
FRAME_WINDOW = st.image([])
FRAME_WINDOW1 = st.image([])
cam0 = cv2.VideoCapture(0) 
cam1 = cv2.VideoCapture(1) 

st.session_state['loggedIn'] = False

def resize(img, size) :
    width = int(img.shape[1]*size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)



def findEncoding(images) :
    imgEncodings = []
    for img in images :
        img = resize(img, 0.50)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_rec.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings
def MarkAttendence(name):
    with open('attendence.csv', 'r+') as f:
        myDatalist =  f.readlines()
        nameList = []
        for line in myDatalist :
            entry = line.split(',')
            nameList.append(entry[0])

        if name not in nameList:
            now = datetime.now()
            timestr = now.strftime('%H:%M')
            f.writelines(f'\n{name}, {timestr}')
            statment = str('welcome to seasia' + name)

for cl in myList :
    curimg = cv2.imread(f'{path}/{cl}')
    employeeImg.append(curimg)
    employeeName.append(os.path.splitext(cl)[0])

EncodeList = findEncoding(employeeImg)

import pandas as pd
import sqlite3
from datetime import datetime

# datetime object containing current date and time
def employee_creation(serial):
    data_base = db_name
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    
    
    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:
    
        cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
        params = ('Initial',time, 'Initial_str')



        cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)

        con.commit()
        con.close()


#detect employee
def employee_attendance(serial,status,image_string):
    
    serial = int(serial)    
    
    data_base = db_name
    
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
   
    seriall = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    df = employee_fetch(serial)
    if df['Status'].iloc[-1]!=status:

        print(df)
        if df['Status'].iloc[-1]!=status:
            params = (status,time, image_string)



            cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
            st.warning(f'attendance {status} {seriall}')
        con.commit()
        con.close()


#fetch employee
def employee_fetch(serial):
    data_base = db_name
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df

st.session_state['loggedIn'] = False
st.session_state['ADD'] = False



def employee_details_creation(serial,name,department):
    db_name2 = 'Employee_details'
    print(db_name2)
    data_base = db_name2
    t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    
    #cur.execute(f"CREATE TABLE t(Department text,Emp_ID integer primary key, Emp_Name text not null)")



    cur.execute('''INSERT INTO t(Department, Emp_ID, Emp_Name)
                   VALUES(?,?,?)''', (department, serial, name))
    con.commit()
    con.close()
employee_details_creation(3598,'seema','ML')


def employee_details_fetch(serial = None,name= None,department = None):
    db_name2 = 'Employee_details'
    print(db_name2)
    data_base = db_name2
    t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    result_df = pd.DataFrame()
    df = pd.read_sql_query(f'SELECT * FROM t', con)
    con.commit()
    con.close()    
    
    
    
    if serial :
        result_df = df.loc[df['Emp_ID'] == serial]
    elif (name != None) and (department != None) :
        result_df = df.loc[(df['Emp_Name'] == name) & (df['Department'] == department)] 
        
        
    return  result_df    
        
        


employee_details_creation(3598,'seema','ML')




def main():
    st.header("Attendence")

    pages = {

        "Streaming ": employ_recog,        
        "Admin ": Admin,
        "Attendance": Check_attendance
    }
    page_titles = pages.keys()

    page_title = st.sidebar.selectbox(
        "Choose the app mode",
        page_titles,
    )
    st.subheader(page_title)

    page_func = pages[page_title]
    page_func()

    st.sidebar.markdown(
        """
---
    """,  # noqa: E501
        unsafe_allow_html=True,
    )
    
    







def emprec0(img):    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img)

    encodeFacesInFrame = face_rec.face_encodings(img, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            data = im.fromarray(img)
            data.save("data.jpg")

            image = open('data.jpg', 'rb')
            image_read = image.read()
            bs4str = base64.b64encode(image_read)


            #bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'IN',bs4str)   
    return img   


def emprec1(img):    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img)

    encodeFacesInFrame = face_rec.face_encodings(img, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            data = im.fromarray(img)
            data.save("data.jpg")

            image = open('data.jpg', 'rb')
            image_read = image.read()
            bs4str = base64.b64encode(image_read)


            #bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'OUT',bs4str)   
    return img   



        
def Admin():

    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.checkbox('Login') :


            if user == 'b' and passwd == 'b' :
                st.session_state['loggedIn'] = True

                st.success("Logged In as {}".format(user))
                
                # Tasks For Only Logged In Users
                if st.checkbox("ADD_USER"):  
                    st.session_state['ADD'] = True                              
                    employee_seriel_no=st.text_input('Employee Seriel no')
                    st.write('Employee Seriel no ',employee_seriel_no)
                    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])
                    if img_file_buffer is not None:
                        bytes_data = img_file_buffer.getvalue()
                        cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
                        #cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
                        img = np.array(cv2_img)
                        cv2.imwrite(f'employee images/{employee_seriel_no}.jpg', img)
                        employee_creation(employee_seriel_no)
                        st.image(img)

    
def employ_recog():
        st.title("Webcam Application")
        run = st.checkbox('Run')
        while run:
            ret0, frame0 = cam0.read()  

            #print(frame)       
            img0 = emprec0(frame0)


            ret1, frame1 = cam1.read()  

            #print(frame)       
            img1 = emprec1(frame1)            
            FRAME_WINDOW.image(img0)
            FRAME_WINDOW1.image(img1)

        
def Check_attendance():
    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.checkbox('Login') :
            if user == 'b' and passwd == 'b' :
                st.session_state['loggedIn'] = True
                st.success("Logged In as {}".format(user))
                if st.checkbox('check attendence'):
                    st.session_state['In'] = True
        
                    if st.button('Show attendence'): 
                        Emp_ID  = st.text_input('Employee Seriel no')
                        Emp_name = st.text_input('Employee_name')
                        Department = st.text_input('Department')
                        if Emp_ID:
                            df =  employee_details_fetch(serial = Emp_ID,name = Emp_name,department = Department)
                            st.dataframe(df)
                        elif  Emp_name   
    








if __name__ == "__main__":
    import os



    main()


In [70]:
db_name2 = 'Employee_details'

def employee_details_creation(serial,name,department):
    db_name2 = 'Employee_details'
    print(db_name2)
    data_base = db_name2
    t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    
    #cur.execute(f"CREATE TABLE t(Department text,Emp_ID integer primary key, Emp_Name text not null)")



    cur.execute('''INSERT INTO t(Department, Emp_ID, Emp_Name)
                   VALUES(?,?,?)''', (department, serial, name))
    #con.commit()
    con.close()
    con.commit()



def employee_details_fetch(serial = None,name= None,department = None):
    db_name2 = 'Employee_details'
    print(db_name2)
    data_base = db_name2
    t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db', timeout=2)
    #cur = con.cursor()
    result_df = pd.DataFrame()
    df = pd.read_sql_query(f'SELECT * FROM t', con)
    #con.commit()
    con.close()   
    con.commit()
    
    
    
    if serial :
        result_df = df.loc[df['Emp_ID'] == serial]
    elif (name != None) and (department != None) :
        result_df = df.loc[(df['Emp_Name'] == name) & (df['Department'] == department)] 
        
        
    return  result_df    
        
        


   

employee_details_creation(3598,'seema','ML')

Employee_details


OperationalError: database is locked

In [63]:

con

In [69]:
employee_details_creation(3598,'seema','ML')

Employee_details


OperationalError: database is locked

In [51]:
conn = sqlite3.connect(db_name2)

def select_all_tasks():
    
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    db_name2 = 'Employee_details'
    data_base = db_name2
    #t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()    

    #cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    df = pd.read_sql_query(f'SELECT * FROM t', con)
    print(df)
    rows = cur.fetchall()
    print(rows)
    for row in rows:
        print(row[0])
select_all_tasks()

         Department  Emp_ID Emp_Name
0  Machine Learning       1    Seema
1            Design       2  Navdeep
[]


In [53]:
conn = sqlite3.connect(db_name2)

def select_all_tasks():
    
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    db_name2 = 'Employee_details'
    data_base = db_name2
    #t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()    

    cur.execute("drop table serial")
#     df = pd.read_sql_query(f'SELECT * FROM t', con)
#     print(df)
#     rows = cur.fetchall()
#     print(rows)
#     for row in rows:
#         print(row[0])
# select_all_tasks()

In [54]:
select_all_tasks()

OperationalError: database is locked

In [3]:
import pandas as pd
import sqlite3
from datetime import datetime

In [4]:
db_name2 = 'Employee_detailss'
print(db_name2)
data_base = db_name2

con = sqlite3.connect(f'{data_base}.db')
cur = con.cursor()

cur.execute(f"CREATE TABLE t(Department text,Emp_ID integer , Emp_Name text )")

Employee_detailss


In [5]:
con.commit()
con.close()

In [21]:

data_base = 'Employee_details'
con = sqlite3.connect(f'{data_base}.db')
now = datetime.now()
time = now.strftime("%d/%m/%Y %H:%M:%S")
cur = con.cursor()
cur.execute(f"CREATE TABLE Employee_detail(Emp_ID int,Emp_Name text, Department text)")

In [ ]:
def employee_creation(serial,name,department):
    data_base = 'Employee_detailss'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    
    
    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:
    
        cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
        params = ('Initial',time, 'Initial_str')



        cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
        cur.execute(f"INSERT INTO Employee_details VALUES (?,?, ?)", (serial,name,department))
        con.commit()
        con.close()

In [8]:
pd.read_sql_query(f'SELECT * FROM Employee_details', con)

,Emp_ID,Emp_Name,Department
0,567,akakak,ML
1,101,see,Machine Learning


In [15]:
def employee_fetch(serial):
    data_base = 'Employee_detailss'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df

In [11]:
employee_fetch(567)

    Status                 time  image_sting
0  Initial  29/08/2022 14:56:39  Initial_str


,Status,time,image_sting
0,Initial,29/08/2022 14:56:39,Initial_str


In [19]:

def employee_details_fetch(serial = None,name= None,department = None):
    db_name2 = 'Employee_detailss'
    print(db_name2)
    data_base = db_name2
    t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    result_df = pd.DataFrame()
    df = pd.read_sql_query(f'SELECT * FROM Employee_details', con)
    con.commit()
    con.close()    
    print(serial)
    print(df)
    
    if serial:
        result_df = df.loc[df['Emp_ID'] == serial]
        print(result_df)

    elif (name != None) and (department != None) :
        result_df = df.loc[(df['Emp_Name'] == name) & (df['Department'] == department)] 
        
        
    return  result_df    

In [20]:
df = employee_details_fetch((567))

Employee_detailss
567
   Emp_ID Emp_Name        Department
0     567   akakak                ML
1     101      see  Machine Learning
2    1001      cma                ML
   Emp_ID Emp_Name Department
0     567   akakak         ML


In [18]:
employee_fetch(df['Emp_ID'].iloc[0])

    Status                 time  image_sting
0  Initial  29/08/2022 14:56:39  Initial_str


,Status,time,image_sting
0,Initial,29/08/2022 14:56:39,Initial_str


In [23]:
data_base = 'Employee_details'

con = sqlite3.connect(f'{data_base}.db')
now = datetime.now()
time = now.strftime("%d/%m/%Y %H:%M:%S")
cur = con.cursor()


cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
listof_table = (cur.fetchall())


z = []
for s in listof_table:
    z.append(s[0])

In [24]:
z

['Employee_detail', 'ID_3291', 'ID_3598']

In [ ]:
 option = st.selectbox('select department',('Machine Learning', 'Design', 'Digital marketing'))



In [ ]:
import urllib.request
from pathlib import Path
from typing import List, NamedTuple, Optional
import os
import pandas as pd
from datetime import  datetime,date
from PIL import Image as im
import cv2

import numpy as np

import streamlit as st

import base64
db_name = 'face_rec_attendence1'
##
import face_recognition as face_rec
import cv2
import shutil
path = 'employee images'
employeeImg = []

employeeName = []
myList = os.listdir(path)
filename = 'click'
FRAME_WINDOW = st.image([])
FRAME_WINDOW1 = st.image([])
cam0 = cv2.VideoCapture(0) 
cam1 = cv2.VideoCapture(1) 

st.session_state['loggedIn'] = False

def resize(img, size) :
    width = int(img.shape[1]*size)
    height = int(img.shape[0] * size)
    dimension = (width, height)
    return cv2.resize(img, dimension, interpolation= cv2.INTER_AREA)



def findEncoding(images) :
    imgEncodings = []
    for img in images :
        img = resize(img, 0.50)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encodeimg = face_rec.face_encodings(img)[0]
        imgEncodings.append(encodeimg)
    return imgEncodings
def MarkAttendence(name):
    with open('attendence.csv', 'r+') as f:
        myDatalist =  f.readlines()
        nameList = []
        for line in myDatalist :
            entry = line.split(',')
            nameList.append(entry[0])

        if name not in nameList:
            now = datetime.now()
            timestr = now.strftime('%H:%M')
            f.writelines(f'\n{name}, {timestr}')
            statment = str('welcome to seasia' + name)

for cl in myList :
    curimg = cv2.imread(f'{path}/{cl}')
    employeeImg.append(curimg)
    employeeName.append(os.path.splitext(cl)[0])

EncodeList = findEncoding(employeeImg)

import pandas as pd
import sqlite3
from datetime import datetime

# datetime object containing current date and time
def employee_creation(seriall,name,department,img):
    data_base = 'Employee_details'
    serial = 'ID_' + str(seriall)
    con = sqlite3.connect(f'{data_base}.db')
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
    cur = con.cursor()


    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    
    
    z = []
    for s in listof_table:
        z.append(s[0])

    if serial not in z:

    
        cur.execute(f"CREATE TABLE {serial}(Status text,time TIMESTAMP, image_sting text)")
        params = ('Initial',time, 'Initial_str')



        cur.execute(f"INSERT INTO {serial} VALUES (?,?, ?)", params)
        cur.execute(f"INSERT INTO Employee_detail VALUES (?,?, ?)", (seriall,name,department))
        con.commit()
        con.close()
        cv2.imwrite(f'employee images/{seriall}.jpg', img)
    else:
        st.write('Employee already exists')
#detect employee
def employee_attendance(serial,status,image_string):
    
    serial = int(serial)    
    
    data_base = 'Employee_details'
    
    now = datetime.now()
    time = now.strftime("%d/%m/%Y %H:%M:%S")
   
    seriall = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    listof_table = (cur.fetchall())
    df = employee_fetch(serial)
    if df['Status'].iloc[-1]!=status:

        print(df)
        if df['Status'].iloc[-1]!=status:
            params = (status,time, image_string)



            cur.execute(f"INSERT INTO {seriall} VALUES (?,?, ?)", params)
            st.warning(f'attendance {status} {seriall}')
        con.commit()
        con.close()


#fetch employee
def employee_fetch(serial):
    data_base = 'Employee_details'
    serial = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()


    df = pd.read_sql_query(f'SELECT * FROM {serial}', con)
    print(df)
    return df

st.session_state['loggedIn'] = False
st.session_state['ADD'] = False






def employee_details_fetch(serial = None,name= None,department = None):
    db_name2 = 'Employee_details'
    print(db_name2)
    data_base = db_name2
    t = 'ID_' + str(serial)
    con = sqlite3.connect(f'{data_base}.db')
    cur = con.cursor()
    result_df = pd.DataFrame()
    df = pd.read_sql_query(f'SELECT * FROM Employee_detail', con)
    con.commit()
    con.close()    
    print(serial)
    print(df)
    
    if serial:
        result_df = df.loc[df['Emp_ID'] == int(serial)]
        print(result_df)

    elif (name != None) and (department != None) :
        result_df = df.loc[(df['Emp_Name'] == name) & (df['Department'] == department)] 
        
        
    return  result_df    
        
        




def main():
    st.header("Attendance")
    image = im.open('seasia-infotech-largex5-logo.png')

    st.image(image, caption='Welcome to Seasia')
    

    pages = {

        "Streaming ": employ_recog,        
        "Admin ": Admin,
        "Attendance": Check_attendance
    }
    page_titles = pages.keys()

    page_title = st.sidebar.selectbox(
        "Choose the app mode",
        page_titles,
    )
    st.subheader(page_title)

    page_func = pages[page_title]
    page_func()

    st.sidebar.markdown(
        """
---
    """,  # noqa: E501
        unsafe_allow_html=True,
    )
    
    







def emprec0(img):    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img)

    encodeFacesInFrame = face_rec.face_encodings(img, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        print(EncodeList,encodeFace)
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            data = im.fromarray(img)
            data.save("data.jpg")

            image = open('data.jpg', 'rb')
            image_read = image.read()
            bs4str = base64.b64encode(image_read)


            #bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'IN',bs4str)   
    return img   


def emprec1(img):    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    
    facesInFrame = face_rec.face_locations(img)

    encodeFacesInFrame = face_rec.face_encodings(img, facesInFrame)


    for encodeFace, faceloc in zip(encodeFacesInFrame, facesInFrame) :
        matches = face_rec.compare_faces(EncodeList, encodeFace)
        facedis = face_rec.face_distance(EncodeList, encodeFace)
        print(facedis)
        if min(facedis) < 0.5:
            matchIndex = np.argmin(facedis)

            print(matchIndex)


            name = employeeName[matchIndex].upper()
            data = im.fromarray(img)
            data.save("data.jpg")

            image = open('data.jpg', 'rb')
            image_read = image.read()
            bs4str = base64.b64encode(image_read)


            #bs4str = base64.b64encode(img)
#             y1, x2, y2, x1 = faceloc
#             y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4

#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
#             cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), cv2.FILLED)
#             cv2.putText(img, name, (x1+6, y2-6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            top, right, bottom, left = faceloc
            cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(img, name,  (left + 6, bottom - 6), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
            name = name.replace(' ','') 
            employee_attendance(name,'OUT',bs4str)   
    return img   



        
def Admin():

    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.checkbox('Login') :


            if user == 'b' and passwd == 'b' :
                st.session_state['loggedIn'] = True

                st.success("Logged In as {}".format(user))
                
                # Tasks For Only Logged In Users
                if st.checkbox("ADD_USER"):  
                    st.session_state['ADD'] = True                              
                    employee_seriel_no=st.text_input('Employee Seriel no')
                    name =st.text_input('Name')

                    option = st.selectbox('select department',('Machine Learning', 'Design', 'Digital marketing'))

                    department=option
                    st.write('Employee Seriel no ',employee_seriel_no)
                    img_file_buffer = st.file_uploader("Upload Image of Employee", type=["png","jpg","jpeg"])
                    if img_file_buffer is not None:
                        bytes_data = img_file_buffer.getvalue()
                        cv2_img = cv2.imdecode(np.frombuffer(bytes_data, np.uint8), cv2.IMREAD_COLOR)  
                        #cv2_img = cv2.cvtColor(cv2_img, cv2.COLOR_BGR2RGB)
                        img = np.array(cv2_img)

                        if st.checkbox("ADD_USER_TO_DB"):  
                            employee_creation(employee_seriel_no,name,department,img)
                            st.image(img)

    
def employ_recog():
        st.title("Webcam Application")
        run = st.checkbox('Run')
        while run:
            ret0, frame0 = cam0.read()  

            #print(frame)       
            img0 = emprec0(frame0)


            ret1, frame1 = cam1.read()  

            #print(frame)       
            img1 = emprec1(frame1)            
            FRAME_WINDOW.image(img0)
            FRAME_WINDOW1.image(img1)

        
def Check_attendance():
    if st.session_state['loggedIn'] == False:
        user = st.text_input('Username')
        passwd = st.text_input('Password',type='password')
        if st.checkbox('Login') :
            if user == 'b' and passwd == 'b' :
                st.session_state['loggedIn'] = True
                st.success("Logged In as {}".format(user))
                if st.checkbox('check attendence'):
                    st.session_state['In'] = True
        

                    Emp_ID  = st.text_input('Employee Seriel no')
                    Emp_name = st.text_input('Employee_name')



                    option = st.selectbox('select department',('Machine Learning', 'Design', 'Digital marketing'))

                    Department = option
                    if st.checkbox('show attendence'):
                        dfr = pd.DataFrame()

                        df =  employee_details_fetch(serial = Emp_ID,name = Emp_name,department = Department)
                        print(df) 
                        if  len(df) >0:

                            dfr = employee_fetch(df['Emp_ID'].iloc[0])

                            
                            st.write(str(df['Emp_Name'].iloc[0]))
                            st.dataframe(dfr)
                            today = date.today()
                           
                            start_date = st.date_input('Start date', today)
                            end_date = st.date_input('End date', today)
                       

                            dfr['time']= pd.to_datetime(dfr['time'])            

                            mask = (dfr['time'].dt.date >= start_date) & (dfr['time'].dt.date <= end_date)
                            df2 = dfr.loc[mask]
     

                            st.dataframe(df2)     

                        else :
                            st.write('Kindly check employe details')  
    








if __name__ == "__main__":
    import os



    main()


In [1]:
import pandas as pd

In [20]:
df = pd.read_csv('df2.csv')

In [3]:
df

,Unnamed: 0,Status,time,image_sting
0,0,Initial,2022-08-29 15:51:21,Initial_str
1,1,IN,2022-08-29 15:55:47,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
2,2,OUT,2022-08-29 16:04:47,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...
3,3,IN,2022-08-29 16:04:52,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...


In [5]:
df['Status_OUT'] = df['Status'] 
df['time_out'] = df['time'] 

In [7]:
df['Status_OUT'] = df['Status_OUT'].shift(-1)
df['time_out'] = df['time_out'].shift(-1)

In [22]:
import numpy as np
def work (df):
    df['Status_OUT'] = df['Status'] 
    df['time_out'] = df['time'] 
    df['Status_OUT'] = df['Status_OUT'].shift(-1)
    df['time_out'] = df['time_out'].shift(-1)
    df = df.loc[((df['Status']  == 'IN') & (df['Status_OUT']  == 'OUT')) | (df['Status']  == 'IN')]
    df['time_out'] = pd.to_datetime(df['time_out'])
    df['time'] = pd.to_datetime(df['time'])
    df['Working hours'] = (df['time_out']-df['time'])
    df = df[['Status','time','image_sting','Status_OUT','time_out','Working hours']]
    return df

In [23]:
work(df)

C:\Users\vashistanush\AppData\Local\Temp\ipykernel_7964\475283390.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_out'] = pd.to_datetime(df['time_out'])
C:\Users\vashistanush\AppData\Local\Temp\ipykernel_7964\475283390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = pd.to_datetime(df['time'])
C:\Users\vashistanush\AppData\Local\Temp\ipykernel_7964\475283390.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

,Status,time,image_sting,Status_OUT,time_out,Working hours
1,IN,2022-08-29 15:55:47,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,OUT,2022-08-29 16:04:47,0 days 00:09:00
3,IN,2022-08-29 16:04:52,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,NaN,NaT,NaT


In [13]:
df['time_out'] = pd.to_datetime(df['time_out'])
df['time'] = pd.to_datetime(df['time'])

In [17]:
df['Working hours'] = (df['time_out']-df['time'])

In [18]:
df

,Unnamed: 0,Status,time,image_sting,Status_OUT,time_out,Working hours
0,0,Initial,2022-08-29 15:51:21,Initial_str,IN,2022-08-29 15:55:47,0 days 00:04:26
1,1,IN,2022-08-29 15:55:47,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,OUT,2022-08-29 16:04:47,0 days 00:09:00
2,2,OUT,2022-08-29 16:04:47,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,IN,2022-08-29 16:04:52,0 days 00:00:05
3,3,IN,2022-08-29 16:04:52,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgH...,NaN,NaT,NaT


In [ ]:


dfz = df3[['time_in','Working hours']]
dfz['Working hours'] = dfz['Working hours'].astype(int)
dfz.index = dfz['time_in']
st.bar_chart(dfz)